## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-23-15-06-49 +0000,world,Shipwreck of Revolutionary War frigate resurfa...,https://www.washingtonpost.com/world/2025/07/2...
1,2025-07-23-15-00-23 +0000,world,Protests flare in Ukraine as Zelensky reins in...,https://www.washingtonpost.com/world/2025/07/2...
2,2025-07-23-11-00-00 +0000,world,"Bound for Beijing, E.U. leaders face two-front...",https://www.washingtonpost.com/world/2025/07/2...
3,2025-07-23-10-00-03 +0000,world,Volkswagen kept a dark secret in the Amazon. T...,https://www.washingtonpost.com/world/interacti...
4,2025-07-23-06-00-07 +0000,world,Ukraine’s top commander asks Trump to help tak...,https://www.washingtonpost.com/world/2025/07/2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2285/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,3
2,war,2
18,two,2
9,ukraine,2
17,face,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-07-23-11-00-00 +0000,world,"Bound for Beijing, E.U. leaders face two-front...",https://www.washingtonpost.com/world/2025/07/2...,14
4,2025-07-23-06-00-07 +0000,world,Ukraine’s top commander asks Trump to help tak...,https://www.washingtonpost.com/world/2025/07/2...,12
1,2025-07-23-15-00-23 +0000,world,Protests flare in Ukraine as Zelensky reins in...,https://www.washingtonpost.com/world/2025/07/2...,8
0,2025-07-23-15-06-49 +0000,world,Shipwreck of Revolutionary War frigate resurfa...,https://www.washingtonpost.com/world/2025/07/2...,8
3,2025-07-23-10-00-03 +0000,world,Volkswagen kept a dark secret in the Amazon. T...,https://www.washingtonpost.com/world/interacti...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,14,2025-07-23-11-00-00 +0000,world,"Bound for Beijing, E.U. leaders face two-front...",https://www.washingtonpost.com/world/2025/07/2...
4,9,2025-07-23-06-00-07 +0000,world,Ukraine’s top commander asks Trump to help tak...,https://www.washingtonpost.com/world/2025/07/2...
3,8,2025-07-23-10-00-03 +0000,world,Volkswagen kept a dark secret in the Amazon. T...,https://www.washingtonpost.com/world/interacti...
6,7,2025-07-23-03-35-43 +0000,world,"U.S. deportees, freed from Salvadoran prison, ...",https://www.washingtonpost.com/world/2025/07/2...
9,7,2025-07-22-20-07-40 +0000,world,Syrian American executed during violence in Sw...,https://www.washingtonpost.com/world/2025/07/2...
0,6,2025-07-23-15-06-49 +0000,world,Shipwreck of Revolutionary War frigate resurfa...,https://www.washingtonpost.com/world/2025/07/2...
1,6,2025-07-23-15-00-23 +0000,world,Protests flare in Ukraine as Zelensky reins in...,https://www.washingtonpost.com/world/2025/07/2...
7,6,2025-07-22-21-38-09 +0000,world,Detention of two Italians at ‘Alligator Alcatr...,https://www.washingtonpost.com/world/2025/07/2...
8,5,2025-07-22-20-27-16 +0000,world,Afghans promised a home in U.S. may face repat...,https://www.washingtonpost.com/world/2025/07/2...
5,4,2025-07-23-04-00-00 +0000,world,China’s strategy? Let Trump cook.,https://www.washingtonpost.com/world/2025/07/2...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
